In [1]:
import psycopg2 as pg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
columns_business = [
'business_id',
# 'name',
# 'address',
'city',
'state',
'postal_code',
'latitude',
'longitude',
# 'stars',
'review_count',
'attributes_goodforkids',
#'categories',   further processing
'is_open',
# 'hours_monday',
# 'hours_tuesday',
# 'hours_wednesday',
# 'hours_thursday',
# 'hours_friday',
# 'hours_saturday',
# 'hours_sunday',
'attributes_restaurantsreservations',
# 'attributes_goodformeal',
# 'attributes_businessparking',
'attributes_caters',
'attributes_noiselevel',
'attributes_restaurantstableservice',
'attributes_restaurantstakeout',
'attributes_restaurantspricerange2',
'attributes_outdoorseating',
'attributes_bikeparking',
# 'attributes_ambience',
'attributes_hastv',
'attributes_wifi',
'attributes_alcohol',
'attributes_restaurantsattire',
'attributes_restaurantsgoodforgroups',
'attributes_restaurantsdelivery',
'attributes_businessacceptscreditcards',
'attributes_businessacceptsbitcoin',
#'attributes_byappointmentonly',
#'attributes_acceptsinsurance',
# 'attributes_music',
'attributes_goodfordancing',
'attributes_coatcheck',
'attributes_happyhour',
# 'attributes_bestnights',
'attributes_wheelchairaccessible',
'attributes_dogsallowed',
# 'attributes_byobcorkage',
'attributes_drivethru',
'attributes_smoking',
#'attributes_agesallowed',
#'attributes_hairspecializesin',
#'attributes_corkage',
#'attributes_byob',
#'attributes_dietaryrestrictions', further processing
#'attributes_open24hours',
#'attributes_restaurantscounterservice',
'restaurant']

In [3]:
list_as_string =', '.join(columns_business)

In [4]:
con = pg.connect(database="postgres", user="flatiron_user_1", password="K*%4t3VK0ab%gn", host="34.74.239.44", port="5432")
cur = con.cursor()

u_cutoff=87
l_cutoff=10

In [5]:

cur.execute(f"SELECT {list_as_string} FROM business WHERE restaurant IS true AND REVIEW_COUNT < {u_cutoff} AND REVIEW_COUNT > {l_cutoff}")
dataframe=cur.fetchall()

In [6]:
dataframe=pd.DataFrame(dataframe)
dataframe.columns=columns_business


In [7]:
# plt.figure(figsize=(16,10))
# sns.heatmap(dataframe.isna())

In [8]:
np.mean(dataframe.isnull(), axis=0)

business_id                              0.000000
city                                     0.000000
state                                    0.000000
postal_code                              0.000000
latitude                                 0.000000
longitude                                0.000000
review_count                             0.000000
attributes_goodforkids                   0.259743
is_open                                  0.000000
attributes_restaurantsreservations       0.268462
attributes_caters                        0.414482
attributes_noiselevel                    0.350843
attributes_restaurantstableservice       0.761391
attributes_restaurantstakeout            0.153357
attributes_restaurantspricerange2        0.050906
attributes_outdoorseating                0.218061
attributes_bikeparking                   0.213781
attributes_hastv                         0.290473
attributes_wifi                          0.318784
attributes_alcohol                       0.298474


In [9]:
dataframe.nunique()

business_id                              37618
city                                       552
state                                       11
postal_code                               7175
latitude                                 34993
longitude                                34729
review_count                                76
attributes_goodforkids                       3
is_open                                      2
attributes_restaurantsreservations           3
attributes_caters                            3
attributes_noiselevel                        9
attributes_restaurantstableservice           3
attributes_restaurantstakeout                3
attributes_restaurantspricerange2            5
attributes_outdoorseating                    3
attributes_bikeparking                       3
attributes_hastv                             3
attributes_wifi                              7
attributes_alcohol                           7
attributes_restaurantsattire                 7
attributes_re

In [10]:
dataframe.attributes_restaurantsreservations.value_counts()
dataframe.attributes_smoking.value_counts()

u'no'         762
u'outdoor'    636
u'yes'        173
'no'            8
'outdoor'       4
None            4
'yes'           1
Name: attributes_smoking, dtype: int64

In [11]:
list_to_strip=[
    'attributes_alcohol',
    'attributes_restaurantsattire',
    'attributes_wifi',
    'attributes_smoking',
    'attributes_noiselevel',
              ]
dataframe2=dataframe.copy()
for col in list_to_strip:
    dataframe2[col]=dataframe2[col].str.strip("u\'")

In [12]:
for col in columns_business:
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='None')
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='none')


/home/will/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [79]:
dataframe2.attributes_restaurantstakeout.nunique(dropna=False)

4

In [68]:
dataframe2.attributes_restaurantstakeout.value_counts(dropna=False)


True     29382
NaN       5786
False     2450
Name: attributes_restaurantstakeout, dtype: int64

,business_id,city,state,postal_code,latitude,longitude,review_count,attributes_goodforkids,is_open,attributes_restaurantsreservations,...,attributes_businessacceptscreditcards,attributes_businessacceptsbitcoin,attributes_goodfordancing,attributes_coatcheck,attributes_happyhour,attributes_wheelchairaccessible,attributes_dogsallowed,attributes_drivethru,attributes_smoking,restaurant


In [15]:
dataframe2.nunique()

business_id                              37618
city                                       552
state                                       11
postal_code                               7175
latitude                                 34993
longitude                                34729
review_count                                76
attributes_goodforkids                       3
is_open                                      2
attributes_restaurantsreservations           3
attributes_caters                            3
attributes_noiselevel                        5
attributes_restaurantstableservice           3
attributes_restaurantstakeout                3
attributes_restaurantspricerange2            5
attributes_outdoorseating                    3
attributes_bikeparking                       3
attributes_hastv                             3
attributes_wifi                              4
attributes_alcohol                           3
attributes_restaurantsattire                 4
attributes_re

In [16]:
u_cutoff=np.percentile(review_counts, 85)
l_cutoff=10  #np.percentile(review_counts, 20)
l_cutoff,u_cutoff

NameError: name 'review_counts' is not defined

In [ ]:
cur.execute(f"SELECT REVIEW_COUNT FROM BUSINESS WHERE restaurant IS true AND REVIEW_COUNT < {u_cutoff} AND REVIEW_COUNT > {l_cutoff}")

In [ ]:
review_counts2=cur.fetchall()

In [ ]:
len(review_counts2)

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(review_counts2);

In [ ]:
con.close()